In [1]:
from skshapes.morphing import ElasticMetric, RigidMotion, ElasticMetric2
from skshapes.loss import OptimalTransportLoss, LandmarkLoss, NearestNeighborsLoss
from skshapes.data import read
from skshapes.tasks import DistanceMatrix, Registration, Registration2
import torch

In [1]:
## Rigid alignment (with landmarks)

from skshapes.data import read
from skshapes.tasks import Registration
from skshapes.morphing import RigidMotion
from skshapes.loss import LandmarkLoss
from skshapes.optimization import LBFGS
from skshapes.data import read

import torch
import os

targetfolder = "data/SCAPE_low_resolution_aligned"
datafolder = "data/SCAPE_low_resolution"

files = os.listdir(datafolder)
reference = read(datafolder + "/" + files[0])
rest = [read(datafolder + "/" + file) for file in files[1:]]

N = reference.points.shape[0]
reference.landmarks = torch.arange(N, dtype=torch.int64)
reference.to_pyvista().save(targetfolder + "/" + files[0])

for i, mesh in enumerate(rest):
    print("Aligning " + files[i + 1] + " to " + files[0] + "...")
    mesh.landmarks = torch.arange(N, dtype=torch.int64)

    r = Registration(
        model=RigidMotion(),
        loss=LandmarkLoss(p=2),
        optimizer=LBFGS(),
        verbose=0,
        n_iter=2,
        device="cpu",
    )

    morphed = r.fit_transform(source=mesh, target=reference)
    morphed.to_pyvista().save(targetfolder + "/" + files[i + 1])

Aligning mesh005.ply to mesh047.ply...
Aligning mesh001.ply to mesh047.ply...
Aligning mesh042.ply to mesh047.ply...
Aligning mesh027.ply to mesh047.ply...
Aligning mesh003.ply to mesh047.ply...
Aligning mesh023.ply to mesh047.ply...
Aligning mesh026.ply to mesh047.ply...
Aligning mesh022.ply to mesh047.ply...
Aligning mesh028.ply to mesh047.ply...
Aligning mesh033.ply to mesh047.ply...
Aligning mesh021.ply to mesh047.ply...
Aligning mesh044.ply to mesh047.ply...
Aligning mesh004.ply to mesh047.ply...
Aligning mesh024.ply to mesh047.ply...
Aligning mesh070.ply to mesh047.ply...
Aligning mesh029.ply to mesh047.ply...
Aligning mesh034.ply to mesh047.ply...
Aligning mesh011.ply to mesh047.ply...
Aligning mesh048.ply to mesh047.ply...
Aligning mesh012.ply to mesh047.ply...
Aligning mesh068.ply to mesh047.ply...
Aligning mesh052.ply to mesh047.ply...
Aligning mesh000.ply to mesh047.ply...
Aligning mesh065.ply to mesh047.ply...
Aligning mesh018.ply to mesh047.ply...
Aligning mesh032.ply to m

In [1]:
##### ICP

rotation = 1

from skshapes.data import read, PolyData
from skshapes.loss import NearestNeighborsLoss
from skshapes.morphing import RigidMotion
from skshapes.tasks import Registration
from skshapes.optimization import LBFGS

import torch

# Load a mesh and apply a rotation
mesh = read("data/SCAPE_low_resolution/mesh000.ply")
mesh.to_pyvista().save("source.vtk")

rotation = 5
r = RigidMotion()
parameter = torch.Tensor([[rotation, rotation, rotation], [0, 0, 0]])
newmesh = r.morph(shape=mesh, parameter=parameter).morphed_shape
newmesh.to_pyvista().save("target.vtk")


source = read("source.vtk")
target = read("target.vtk")

source.landmarks = torch.arange(source.points.shape[0], dtype=torch.int64)
target.landmarks = torch.arange(target.points.shape[0], dtype=torch.int64)

r = Registration(
    model=RigidMotion(),
    loss=NearestNeighborsLoss(),
    optimizer=LBFGS(),
    verbose=1,
    n_iter=5,
    device="cpu",
)

morphed = r.fit_transform(source=source, target=target)
morphed.to_pyvista().save("output.vtk")

[pyKeOps] Warning : at least one of the input tensors is not contiguous. Consider using contiguous data arrays to avoid unnecessary copies.
[pyKeOps] Warning : at least one of the input tensors is not contiguous. Consider using contiguous data arrays to avoid unnecessary copies.
[pyKeOps] Warning : at least one of the input tensors is not contiguous. Consider using contiguous data arrays to avoid unnecessary copies.
[pyKeOps] Warning : at least one of the input tensors is not contiguous. Consider using contiguous data arrays to avoid unnecessary copies.
[pyKeOps] Warning : at least one of the input tensors is not contiguous. Consider using contiguous data arrays to avoid unnecessary copies.
[pyKeOps] Warning : at least one of the input tensors is not contiguous. Consider using contiguous data arrays to avoid unnecessary copies.
[pyKeOps] Warning : at least one of the input tensors is not contiguous. Consider using contiguous data arrays to avoid unnecessary copies.
[pyKeOps] Warning : 

In [1]:
# Elastic registration (with landmarks)

from skshapes.data import read
from skshapes.loss import LandmarkLoss
from skshapes.morphing import ElasticMetric
from skshapes.tasks import Registration
from skshapes.optimization import LBFGS

import torch
import os

datafolder = "data/SCAPE_low_resolution_aligned"

source = read(datafolder + "/" + "mesh001.ply")
target = read(datafolder + "/" + "mesh041.ply")

source.landmarks = torch.arange(source.points.shape[0], dtype=torch.int64)
target.landmarks = torch.arange(target.points.shape[0], dtype=torch.int64)

from time import time

start = time()
r = Registration(
    model=ElasticMetric(n_steps=10),
    loss=LandmarkLoss(),
    optimizer=LBFGS(),
    verbose=1,
    n_iter=5,
    regularization=100,
    device="cpu",
)
newshape = r.fit_transform(source=source, target=target)
print(time() - start)

Loss value at iteration 0 : 5.045347213745117
Loss value at iteration 1 : 0.05011105164885521
Loss value at iteration 2 : 0.05011105164885521
Loss value at iteration 3 : 0.05011105164885521
Loss value at iteration 4 : 0.05011105164885521
0.27788662910461426


In [1]:
from skshapes.preprocessing import (
    LandmarkSetter,
    Pipeline,
    Decimation,
    AffineTransformation,
)
from skshapes.data import read
import os

# Read the shapes
datafolder = "/home/GitHub/scikit-shapes-draft/data/SCAPE/decimated/"
shapes = [read(datafolder + "/" + f"aligned_mesh{i:03d}.ply") for i in range(4)]

# Create a pipeline of preprocessing steps
preprocessing = Pipeline(
    steps=[
        LandmarkSetter(landmarks=[[0, 1], [2, 3], [4, 5], [6, 70]], by_indices=True),
        Decimation(target_reduction=0.95),
        AffineTransformation(matrix=[[2, 0, 0], [0, 2, 0], [0, 0, 2]]),
    ]
)

# Apply the preprocessing pipeline to the shapes and get the dataset
dataset = preprocessing.fit_transform(shapes=shapes)

for i in range(len(dataset)):
    print("Size of shape {} : {}".format(i, dataset[i].points.shape))
    print("Landmarks of shape {} : {}".format(i, dataset.landmarks[i]), end="\n\n")

ImportError: cannot import name 'Shape' from 'skshapes.data' (/home/GitHub/scikit-shapes/skshapes/data/__init__.py)

In [1]:
import torch


def barycentric_coordinates(mesh, point):
    # Compute the vectors from the point to the vertices
    vertices = mesh.points
    vectors = vertices - point
    norms = torch.norm(vectors, dim=1)

    tol = 1e-5  # TODO tol can be computed from the mesh resolution ?

    # Test if a vector is zero (that means the point is a vertex of the mesh)
    if torch.sum(vectors.abs().sum(dim=1) < tol):
        indice = torch.where(
            torch.all(torch.eq(vectors, torch.zeros_like(vectors)), dim=1)
        )[0]
        vertex_indice = int(indice[0])
        print(
            "The point is a vertex of the mesh, at the following indice: {}".format(
                vertex_indice
            )
        )
        return (torch.tensor(1.0), torch.tensor([vertex_indice]))

    else:
        # Normalize the vectors
        vectors /= norms.reshape(-1, 1)

        A = mesh.edges[0]
        B = mesh.edges[1]

        cos_angles = (vectors[A] * vectors[B]).sum(dim=1)
        # If cos(angle) = -1 <=> angle = pi, the point is on an edge

        if torch.sum((cos_angles - (-1)).abs() < tol):
            indice = torch.where((cos_angles - (-1)).abs() < tol)[0]
            edge_indice = int(indice[0])
            print(
                "The point is on an edge of the mesh, at the following indice: {}".format(
                    edge_indice
                )
            )

            # Coordinates
            a, b = mesh.edges[:, edge_indice]
            alpha = norms[b] / torch.norm(vertices[a] - vertices[b])
            beta = norms[a] / torch.norm(vertices[a] - vertices[b])
            assert torch.abs(alpha + beta - 1) < tol
            assert (
                (alpha * vertices[a] + beta * vertices[b]) - point
            ).abs().sum() < tol
            return (torch.tensor([alpha, beta]), torch.tensor([a, b]))

        else:
            A = mesh.triangles[0]
            B = mesh.triangles[1]
            C = mesh.triangles[2]

            angles_1 = torch.acos((vectors[A] * vectors[B]).sum(dim=1))
            angles_2 = torch.acos((vectors[B] * vectors[C]).sum(dim=1))
            angles_3 = torch.acos((vectors[C] * vectors[A]).sum(dim=1))

            sum_angles = angles_1 + angles_2 + angles_3
            # If sum_angles is close to 2pi, the point is inside the triangle, or its projection is inside the triangle

            if torch.sum((sum_angles - (2 * torch.pi)).abs() < tol):
                indices = torch.where((sum_angles - (2 * torch.pi)).abs() < tol)[0]
                # If several indices, we must find the one for which the point is inside irself the triangle, and not only its projection
                for i in indices:
                    a, b, c = mesh.triangles[:, i]
                    normals = mesh.triangle_normals[i]

                    if (
                        torch.abs(vectors[a] @ normals.T)
                        + torch.abs(vectors[b] @ normals.T)
                        + torch.abs(vectors[c] @ normals.T)
                        < tol
                    ):
                        indice_triangle = i

                print(
                    "The point is inside a triangle of the mesh, at the following indice: {}".format(
                        indice_triangle
                    )
                )
                # Coordinates
                a, b, c = mesh.triangles[:, indice_triangle]
                mat = torch.cat((vertices[a], vertices[b], vertices[c])).reshape(3, 3).T
                alpha, beta, gamma = torch.inverse(mat) @ point
                assert torch.abs(alpha + beta + gamma - 1) < tol
                assert (
                    (alpha * vertices[a] + beta * vertices[b] + gamma * vertices[c])
                    - point
                ).abs().sum() < tol
                return (torch.tensor([alpha, beta, gamma]), torch.tensor([a, b, c]))

            else:
                print("The point is outside the mesh.")

In [2]:
from skshapes.data import read

mesh = read("data/SCAPE_low_resolution/mesh000.ply")

vertices = mesh.points
e0, e1 = mesh.edges[:, 45]
point_edge = (mesh.points[e0] + 4 * mesh.points[e1]) / 5
a, b, c = mesh.triangles[:, 100]
point_triangle = (mesh.points[a] + 4 * mesh.points[b] + 2 * mesh.points[c]) / 7
point_vertice = mesh.points[42]

point = point_vertice

barycentric_coordinates(mesh, point)

The point is a vertex of the mesh, at the following indice: 42


(tensor(1.), tensor([42]))

In [138]:
from typing import NamedTuple

from skshapes._typing import *


class test(NamedTuple):
    a: int = 0
    b: int = 2


t = test()
t.b

2

In [42]:
import torch

a = torch.load("landmarks.pt")

In [47]:
landmark0 = a[1]

Indices : tensor([[73, 94, 92, 73, 75, 94, 62, 73],
        [ 0,  0,  0,  1,  1,  1,  2,  3]])
Values : tensor([0.2522, 0.3066, 0.4412, 0.0768, 0.6896, 0.2336, 1.0000, 1.0000],
       dtype=torch.float64)


In [1]:
import torch

A = torch.Tensor([[1.0, 0, 0], [4.1, 0, 6.1], [0, 8.2, 0]])
type(A)
B = A.to_sparse_coo()

B.dtype

ModuleNotFoundError: No module named 'torch'

In [7]:
import skshapes

from skshapes.loss import L2Loss
from skshapes.data import read
from skshapes.morphing import ElasticMetric, RigidMotion
from skshapes.tasks import Registration
from skshapes.optimization import LBFGS
import vedo

vedo.settings.default_backend = "vtk"
from vedo.applications import Browser

# Load the shapes
source = read("data/SCAPE_low_resolution/mesh001.ply")
target = read("data/SCAPE_low_resolution/mesh041.ply")


def foo(model, loss, optimizer, regularization):
    r = Registration(
        model=model,
        loss=loss,
        optimizer=optimizer,
        n_iter=5,
        regularization=regularization,
        device="cpu",
    )

    newshape = r.fit_transform(source=source, target=target)

    print(r.parameter.device)
    print(source.device)

    a = model.morph(shape=source, parameter=r.parameter, return_path=True)

    meshes = a.path
    meshes.append(target)

    return meshes

    # plt = Browser([vedo.Mesh(m.to_pyvista()) for m in meshes], resetcam=0, axes=0)  # a vedo.Plotter
    # plt.show().close()


r = Registration(
    model=RigidMotion(),
    loss=L2Loss(),
    optimizer=LBFGS(),
    n_iter=2,
)

print(source.device)
print(target.device)

source = r.fit_transform(source=source, target=target)
source = source.to("cpu")

cpu
cpu


In [11]:
model = ElasticMetric(n_steps=5)
loss = L2Loss()
optimizer = LBFGS()

meshes = foo(model, loss, optimizer, regularization=1000)

vedo.settings.default_backend = "vtk"
Browser([vedo.Mesh(m.to_pyvista()) for m in meshes], resetcam=0, axes=0).show().close()

cpu
cpu


In [4]:
from skshapes.loss import OptimalTransportLoss

loss = OptimalTransportLoss(loss="test")

In [2]:
# Load data
from skshapes.data import read
import pyvista
from skshapes.morphing import ElasticMetric, RigidMotion
from skshapes.loss import NearestNeighborsLoss, OptimalTransportLoss, LandmarkLoss
from skshapes.optimization import LBFGS

import vedo
from vedo.applications import Browser

source = read("data/SCAPE_low_resolution/mesh001.ply")
target = read("data/SCAPE_low_resolution/mesh041.ply")


# ElasticMetric can only be used with polydata
# RigidMotion can be used with all type of shapes


model = RigidMotion()
loss = NearestNeighborsLoss()
optimizer = LBFGS(model.parameters(), lr=0.1)

from skshapes.tasks import Registration


def foo(model, loss, optimizer, regularization):
    r = Registration(
        model=model,
        loss=loss,
        optimizer=optimizer,
        n_iter=5,
        device="cpu",
        regularization=regularization,
    )

    print(loss, model)
    newshape = r.fit_transform(source=source, target=target)

    parameter = r.parameter.detach().cpu().clone()
    n_frames = parameter.shape[0] + 1

    meshes = [source.copy(device="cpu") for _ in range(n_frames)]
    for i in range(n_frames - 1):
        meshes[i + 1].points = meshes[i].points + parameter[i]

    meshes = [vedo.Mesh(mesh.to_pyvista()) for mesh in meshes] + [
        vedo.Mesh(target.to_pyvista())
    ]

    plt = Browser(meshes, resetcam=0, axes=0)  # a vedo.Plotter
    plt.show().close()

In [2]:
# import modules and load a model
import torch
import torchvision.models as models
from torch.profiler import profile, record_function, ProfilerActivity

In [17]:
from skshapes.data import read
import pyvista
from skshapes.morphing import ElasticMetric
from skshapes.loss import L2Loss
from skshapes.optimization import SGD
from skshapes.tasks import Registration

import vedo
from vedo.applications import Browser

source = read("data/SCAPE_low_resolution/mesh001.ply")
target = read("data/SCAPE_low_resolution/mesh041.ply")


def foo(model, loss, optimizer, regularization):
    r = Registration(
        model=model,
        loss=loss,
        optimizer=optimizer,
        n_iter=2,
        device="cuda",
        regularization=regularization,
    )

    newshape = r.fit_transform(source=source, target=target)


model = ElasticMetric()
loss = L2Loss()
optimizer = SGD()


with profile(
    activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], record_shapes=True
) as prof:
    with record_function("model_inference"):
        foo(model, loss, optimizer, regularization=10)


print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))
prof.export_chrome_trace("trace.json")

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                        model_inference         4.62%       2.538ms        67.49%      37.060ms      37.060ms       0.000us         0.00%     649.000us     649.000us             1  
                                Optimizer.step#SGD.step        39.87%      21.896ms        49.25%      27.044ms      13.522ms       0.000us         0.00%     424.000us     212.000us             2  
         

In [22]:
import pyvista
import numpy as np
from pyvista.examples import load_airplane

from skshapes.data import PolyData

mesh = PolyData.from_pyvista(pyvista.Sphere())

n_points = mesh.n_points
n_triangles = mesh.n_triangles
n_edges = mesh.n_edges

n_landmarks = 10

landmarks_type = np.random.choice(["points", "edges", "triangles"], n_landmarks)

import torch

# Create landmarks (coo matrix)

array(['edges', 'points', 'points', 'triangles', 'points', 'points',
       'triangles', 'triangles', 'points', 'edges'], dtype='<U9')